### data preprocessing

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [10]:
import re

In [26]:
df = pd.read_csv('label_drop_duplicate.csv', index_col = 0)
df

,업체명,업종
0,돌비하우스호텔,c16
1,한일관 서울식 육개장,c2
2,오비야 닭고기덮밥,c2
3,츠키지,c2
4,광양참숯불고기,c2
...,...,...
969163,제인 풀빌라,c16
969164,(재)인천교구천주교회유지재단,c8
969165,거창축산,c2
969166,주식회사 포디아트,c19


In [15]:
didi = {'c1':'카페',
        'c2':'외식',
        'c3':'주류/펍',
        'c4':'배달',
        'c5':'편의점',
        'c6':'대형마트',
        'c7':'생필품',
        'c8':'생활서비스',
        'c9':'통신',
        'c10':'쇼핑몰',
        'c11':'온라인쇼핑',
        'c12':'의료/건강',
        'c13':'미용',
        'c14':'교통/자동차',
        'c15':'취미/여가',
        'c16':'여행/숙박',
        'c17':'교육',
        'c18':'세금/공과금',
        'c19':'기타소비',
        'c20':'보험',
        'c21':'미분류'}

In [16]:
data = df.replace(list(didi.keys()), list(didi.values()))

In [18]:
regularized = []
for word in tqdm(data['업체명']):
    regularized.append(re.sub(r"[^가-힣a-zA-Z0-9]", " ", word))
    
data['업체명_r'] = regularized
data['업체명_r'] = data['업체명_r'].str.strip()

for i in range(10):
    data['업체명_r'] = data['업체명_r'].str.replace('  ', ' ')
    data['업체명_r'] = data['업체명_r'].str.replace('   ', ' ')

  0%|          | 0/969168 [00:00<?, ?it/s]

In [25]:
idx = []
for c in range(50):
    idx.append(list(data[data['업체명_r'] == ' '* c].index))
idx = sum(idx, [])
data_ = data.drop(idx).reset_index(drop = True)
data_

,업체명,업종,업체명_r
0,돌비하우스호텔,여행/숙박,돌비하우스호텔
1,한일관 서울식 육개장,외식,한일관 서울식 육개장
2,오비야 닭고기덮밥,외식,오비야 닭고기덮밥
3,츠키지,외식,츠키지
4,광양참숯불고기,외식,광양참숯불고기
...,...,...,...
969159,제인 풀빌라,여행/숙박,제인 풀빌라
969160,(재)인천교구천주교회유지재단,생활서비스,재 인천교구천주교회유지재단
969161,거창축산,외식,거창축산
969162,주식회사 포디아트,기타소비,주식회사 포디아트


In [27]:
data_ = data_[['업체명', '업체명_r','업종']]
data_.head()

,업체명,업체명_r,업종
0,돌비하우스호텔,돌비하우스호텔,여행/숙박
1,한일관 서울식 육개장,한일관 서울식 육개장,외식
2,오비야 닭고기덮밥,오비야 닭고기덮밥,외식
3,츠키지,츠키지,외식
4,광양참숯불고기,광양참숯불고기,외식


In [28]:
data_['업체명_r'].value_counts()[data_['업체명_r'].value_counts() >= 2]

밥               3
다방              2
C D             2
한수개별용달 한수이삿짐    2
상도동입주청소 이사청소    2
               ..
코디              2
C S             2
J J스크린골프        2
밥먹자             2
한전케이디엔 주        2
Name: 업체명_r, Length: 123, dtype: int64

In [29]:
data_drop = data_.drop_duplicates(subset = ['업체명_r', '업종'], keep = 'first').reset_index(drop = True)

In [30]:
data_drop

,업체명,업체명_r,업종
0,돌비하우스호텔,돌비하우스호텔,여행/숙박
1,한일관 서울식 육개장,한일관 서울식 육개장,외식
2,오비야 닭고기덮밥,오비야 닭고기덮밥,외식
3,츠키지,츠키지,외식
4,광양참숯불고기,광양참숯불고기,외식
...,...,...,...
969064,제인 풀빌라,제인 풀빌라,여행/숙박
969065,(재)인천교구천주교회유지재단,재 인천교구천주교회유지재단,생활서비스
969066,거창축산,거창축산,외식
969067,주식회사 포디아트,주식회사 포디아트,기타소비


In [31]:
data_drop['업체명_r'].value_counts()[data_drop['업체명_r'].value_counts() >= 2]

서비스                 2
성진T S               2
B K                 2
청우개발                2
C S                 2
C F                 2
N                   2
좋은느낌                2
조아                  2
믹스                  2
씨티                  2
에스                  2
더풋                  2
아베 홀리               2
러브                  2
봉자                  2
자연                  2
아로마                 2
A                   2
즐겨찾기                2
C D                 2
J S                 2
1                   2
미                   2
대성홀딩스 주 정보통신사업부문    2
J J                 2
한잔할래                2
M J                 2
P G S               2
Name: 업체명_r, dtype: int64

In [32]:
data_drop_ = data_drop.drop_duplicates(subset = ['업체명_r'], keep = False).reset_index(drop = True)

In [33]:
cased = data_drop_.copy()
uncased = data_drop_.copy()
uncased['업체명_r'] = uncased['업체명_r'].str.lower()

In [34]:
cased.to_csv('regularized_cased.csv')
uncased.to_csv('regularized_uncased.csv')

In [85]:
number = data_drop_.copy()
number['업체명_r'] = number['업체명_r'].str.replace('[0-9]+', '#')

/tmp/ipykernel_4901/2737205613.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  number['업체명_r'] = number['업체명_r'].str.replace('[0-9]+', '#')


In [86]:
idx = []
for c in range(50):
    idx.append(list(number[number['업체명_r'] == '#'* c].index))
idx = sum(idx, [])
number_ = number.drop(idx)
number_

,업체명,업체명_r,업종
0,돌비하우스호텔,돌비하우스호텔,여행/숙박
1,한일관 서울식 육개장,한일관 서울식 육개장,외식
2,오비야 닭고기덮밥,오비야 닭고기덮밥,외식
3,츠키지,츠키지,외식
4,광양참숯불고기,광양참숯불고기,외식
...,...,...,...
969006,제인 풀빌라,제인 풀빌라,여행/숙박
969007,(재)인천교구천주교회유지재단,재 인천교구천주교회유지재단,생활서비스
969008,거창축산,거창축산,외식
969009,주식회사 포디아트,주식회사 포디아트,기타소비


In [87]:
number_drop = number_.drop_duplicates(subset = ['업체명_r', '업종'])

In [88]:
number_drop_ = number_drop.drop_duplicates(subset = '업체명_r', keep = False)

In [89]:
number_drop_.to_csv('nonumber.csv')